In [1]:
import os
import numpy as np
import pandas as pd

In [3]:
# Read in Data and then take first 100 entries and use it to play with code

path = os.getcwd() + '/events.csv'
data = pd.read_csv(path)
testData = data.head(2000)

In [16]:
testData.columns

Index(['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
       'location', 'bodypart', 'assist_method', 'situation', 'fast_break'],
      dtype='object')

In [3]:
testData = testData.drop(['id_odsp','id_event', 'sort_order'], axis=1)

Might want to drop time, text, player_in, player_out. More of a maybe is bodypart and fast break.

In [31]:
testData

,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
3,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,sven bender,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
4,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,gokhan tore,NaN,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,81,Robert Lewandowski (Borussia Dortmund) wins a ...,8,NaN,1,Borussia Dortmund,Hamburg SV,robert lewandowski,NaN,NaN,NaN,NaN,NaN,0,1.0,NaN,0,NaN,0
96,82,Attempt missed. Ivan Perisic (Borussia Dortmun...,1,NaN,1,Borussia Dortmund,Hamburg SV,ivan perisic,NaN,NaN,NaN,9.0,2.0,0,15.0,1.0,0,4.0,0
97,82,Foul by Marcell Jansen (Hamburg).,3,NaN,2,Hamburg SV,Borussia Dortmund,marcell jansen,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
98,84,Lukasz Piszczek (Borussia Dortmund) wins a fre...,8,NaN,1,Borussia Dortmund,Hamburg SV,lukasz piszczek,NaN,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0


In [21]:
print(type(testData['shot_place'][0]))

<class 'numpy.float64'>


In [22]:
testData["location"].describe()

count    48.000000
mean      5.645833
std       5.261217
min       1.000000
25%       2.000000
50%       3.000000
75%       9.000000
max      16.000000
Name: location, dtype: float64

In [88]:
testData = data.head(2000)
testData = testData.drop(['id_odsp','id_event', 'sort_order'], axis=1)

In [5]:
count = 0
numGoals = 0
for i in range(2000):
#     if testData['location'][i] == 14.0:
#         print(testData.iloc[i])
    if testData['event_type'][i] == 1:
        count += 1
    if testData['is_goal'][i] == 1:
        numGoals += 1
        
print(count)
print(numGoals)

450
51


The testData['location'][i] is the way to look at one value. Now to do xG need to do an and statement that it is a shot and the certain location.

We have 450 attempts at goal and 51 goals. Going to check to see if blocked shots count in as attempts.

In [6]:
count = 0
for i in range(2000):
    if (testData['event_type'][i] == 1) & (testData['shot_outcome'][i] == 3):
        count += 1
print(count)

102


So Blocked Shots are in the number of total Attempts. Leaning to having Blocked Shots not count in total attempts because it would alter the number of total attempts. Also think it is common practice not to include blocked shots in xG calculations. But going to leave shots that come from area 19, which is "unknown" because they are all goals.

Also get rid of events that are announcements, free kicks won, offsides, and handballs

Free kicks won seem to be a very large proportion of the data

In [89]:
print(testData.shape)
# This deletes wherever shot_outcome = 3
tempData = testData.loc[testData["shot_outcome"] != 3]
testData = tempData
# tempData = testData.loc[testData["location"] != 19]
# testData = tempData
tempData = testData.loc[testData["event_type"] != 0]
testData = tempData
tempData = testData.loc[testData["event_type"] != 8]
testData = tempData
tempData = testData.loc[testData["event_type"] != 9]
testData = tempData
tempData = testData.loc[testData["event_type"] != 10]
testData = tempData
print(testData.shape)

(2000, 19)
(1247, 19)


In [91]:
# Reset the index
testData = testData.reset_index(drop = True)

Shots Come from locations 3-18. So now going to calculate the xG for each shot location

In [92]:
# Get a data set that only has data about shots on it
shotData = testData.loc[testData["event_type"] == 1]

In [93]:
print(shotData.shape)

(348, 19)


Going to want to loop over the entire shot dataset and have if statements about each zone

In [160]:
shotData = shotData.reset_index(drop=True)
shotData.head()

,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,14,Attempt missed. Shinji Kagawa (Borussia Dortmu...,1,12.0,1,Borussia Dortmund,Hamburg SV,shinji kagawa,mario gotze,NaN,NaN,13.0,2.0,0,15.0,1.0,1,1.0,0
2,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,kevin grosskreutz,mario gotze,NaN,NaN,4.0,1.0,1,9.0,2.0,1,1.0,0
3,25,Shinji Kagawa (Borussia Dortmund) hits the rig...,1,12.0,1,Borussia Dortmund,Hamburg SV,shinji kagawa,lukasz piszczek,NaN,NaN,7.0,4.0,0,3.0,1.0,1,1.0,0
4,26,Attempt saved. Mats Hummels (Borussia Dortmund...,1,12.0,1,Borussia Dortmund,Hamburg SV,mats hummels,chris lowe,NaN,NaN,5.0,1.0,0,3.0,3.0,2,1.0,0


Doing it that way is very inefficient so going to make lists instead.

In [176]:
numShots = list()
numGoals = list()
xG = list()

In [162]:
for i in range(3,20):
    print(i)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [177]:
countShots = 0
countGoals = 0

for j in range(3,20):
    countShots = 0
    countGoals = 0
    
    for i in range(shotData.shape[0]):
        if (shotData['location'][i] == j) & (shotData['is_goal'][i] == 0):
            countShots += 1
        if (shotData['location'][i] == j) & (shotData['is_goal'][i] == 1):
            countShots += 1
            countGoals += 1
            
    numShots.append(countShots)
    numGoals.append(countGoals)

In [178]:
for i in range(len(numShots)):
    if(numShots[i] != 0):
        xG.append(numGoals[i]/numShots[i])
    # Going to make zone 19 the average xG
#     elif(numShots[i] == numGoals[i]):
#         xG.append(countGoals/countShots)
    else:
        xG.append(0)


In [186]:
print(xG)

[0.16346153846153846, 0, 0, 0.4, 0.2, 0.0, 0.11538461538461539, 0.16666666666666666, 0.14285714285714285, 0.3333333333333333, 0.4, 0.6, 0.051470588235294115, 0.0, 0.0, 0, 0.14655172413793102]


Note: Zone 19 has an xG of 1.0 since the shots in that zone are goals.

In [185]:
xG[16] = countGoals / countShots

In [184]:
countShots = 0
countGoals = 0
for i in range(len(numShots)):
    countShots += numShots[i]
    countGoals += numGoals[i]

print(countShots)
print(countGoals)

348
51


Now want to assign xG to each event. Only shots will accrue xG so need to code for that

In [123]:
testData['xG'] = 0#testData['player_in'][0] # nan
testData['is_shot'] = 0
testData['is_corner'] = 0
testData.head()

,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,...,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,xG,is_shot,is_corner
0,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,NaN,...,2.0,0,9.0,2.0,1,1.0,0,0,0,0
1,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0,0,0
2,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0,0,0
3,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,sven bender,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0,0,0
4,10,"Corner, Hamburg. Conceded by Lukasz Piszczek.",2,NaN,2,Hamburg SV,Borussia Dortmund,lukasz piszczek,lukasz piszczek,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0,0,0


In [51]:
for j in range(len(numShots)):
    print(j+3)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [215]:
for i in range(testData.shape[0]):
    if (testData['event_type'][i]==1):
        testData['is_shot'][i] = 1
        
        for j in range(len(numShots)):
            if(testData['event_type'][i]==1) & (testData['location'][i] == j+3):
                testData['xG'][i] = xG[j]
                
    if(testData['event_type'][i]==2):
        testData['is_corner'][i] = 1

C:\Users\Joe Yando\AppData\Local\Temp\ipykernel_10996\3943008318.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData['is_shot'][i] = 1
C:\Users\Joe Yando\AppData\Local\Temp\ipykernel_10996\3943008318.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData['is_corner'][i] = 1


In [217]:
testData.shape

(1247, 22)

In [216]:
testData

,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,...,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,xG,is_shot,is_corner
0,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,NaN,...,2.0,0,9.0,2.0,1,1.0,0,0.088675,1,0
1,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,1
2,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,1
3,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,sven bender,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,0
4,10,"Corner, Hamburg. Conceded by Lukasz Piszczek.",2,NaN,2,Hamburg SV,Borussia Dortmund,lukasz piszczek,lukasz piszczek,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,88,John Boye (Rennes) is shown the yellow card.,4,NaN,2,Stade Rennes,Dijon FCO,john boye,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,0
1243,90,Foul by Raphael Caceres (Dijon).,3,NaN,1,Dijon FCO,Stade Rennes,raphael caceres,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,0
1244,90,Foul by Victor Hugo Montano (Rennes).,3,NaN,2,Stade Rennes,Dijon FCO,victor hugo montano,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,0
1245,90,Foul by Yann M'Vila (Rennes).,3,NaN,2,Stade Rennes,Dijon FCO,yann mvila,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,0


This worked! Now every event has the correct xG. A lot of them are zero because most of the events don't add any expected goals
only shots do that.

Might want to create a boolean is_shot variable. This will make creating a single line per game a lot easier.

Now, going to try and combine all the events in one game into a single line

In [126]:
tempData = testData
tempData.head(10)

,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,...,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,xG,is_shot,is_corner
0,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,NaN,...,2.0,0,9.0,2.0,1,1.0,0,0.115385,1,0
1,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,1
2,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,1
3,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,sven bender,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,0
4,10,"Corner, Hamburg. Conceded by Lukasz Piszczek.",2,NaN,2,Hamburg SV,Borussia Dortmund,lukasz piszczek,lukasz piszczek,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,1
5,11,Foul by Gojko Kacar (Hamburg).,3,NaN,2,Hamburg SV,Borussia Dortmund,gojko kacar,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,0
6,13,Foul by Gokhan Tore (Hamburg).,3,NaN,2,Hamburg SV,Borussia Dortmund,gokhan tore,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,0
7,14,Attempt missed. Shinji Kagawa (Borussia Dortmu...,1,12.0,1,Borussia Dortmund,Hamburg SV,shinji kagawa,mario gotze,NaN,...,2.0,0,15.0,1.0,1,1.0,0,0.051471,1,0
8,14,Foul by Gojko Kacar (Hamburg).,3,NaN,2,Hamburg SV,Borussia Dortmund,gojko kacar,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.000000,0,0
9,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,kevin grosskreutz,mario gotze,NaN,...,1.0,1,9.0,2.0,1,1.0,0,0.115385,1,0


In [40]:
tempData['side'][0]

2

In [99]:
tempHomeTeam = ' '
tempAwayTeam = ' '
numGames = 0
boolGame = False

for i in range(tempData.shape[0]):
    if i == 0:
        if(tempData['side'][i] == 1):
            tempHomeTeam = tempData['event_team'][i]
            tempAwayTeam = tempData['opponent'][i]
        else:
            tempAwayTeam = tempData['event_team'][i]
            tempHomeTeam = tempData['opponent'][i]
    
    
    boolGame = (((tempData['side'][i] == 1) & ((tempHomeTeam == tempData['event_team'][i])&(tempAwayTeam == tempData['opponent'][i])))|
               ((tempData['side'][i] == 2) & ((tempAwayTeam == tempData['event_team'][i])&(tempHomeTeam == tempData['opponent'][i]))))
    
    
    # If in same game, Add current row to 
    if(boolGame):
        print(tempHomeTeam)
    # If not in same game, Change the home team and away team
    else:
        
        if(tempData['side'][i] == 1):
            tempHomeTeam = tempData['event_team'][i]
            tempAwayTeam = tempData['opponent'][i]
        else:
            tempAwayTeam = tempData['event_team'][i]
            tempHomeTeam = tempData['opponent'][i]
        # Add one to the number of games
        numGames +=1
    
    # Now make a new row and add to that
    if(boolGame): # Will need to update this statement
        print(tempHomeTeam)
    
        
    print(numGames)

Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dortmund
Borussia Dortmund
0
Borussia Dor

AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Ajaccio
7
AC Ajaccio
AC Aj

Hannover 96
Hannover 96
13
Hannover 96
Hannover 96
13
Hannover 96
Hannover 96
13
Hannover 96
Hannover 96
13
Hannover 96
Hannover 96
13
Hannover 96
Hannover 96
13
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseille
Marseille
14
Marseil

Okay so the last Cell is the overall format to switch between games. Now going to create an array, or dataframe to store each game where numGames will be the Id for the array. Going to have name, numGoals, xG, totalShots. Maybe add subs, numYellowCard, and numRedCards

In [127]:
#colNames = ['HomeTeam', 'AwayTeam', 'HomeGoals', 'AwayGoals', 'HomeXG', 'AwayXG', 'HomeShots', 'AwayShots','HomeCorners', 
            #'AwayCorners', 'HomeSubs','AwaySubs', 'HomeYC', 'AwayYC', 'HomeRC', 'AwayRC']

colNames = ['HomeGoals', 'AwayGoals', 'HomeXG', 'AwayXG', 'HomeShots', 'AwayShots', 'HomeCorners', 'AwayCorners', 'HomeSubs', 
            'AwaySubs', 'HomeYC', 'AwayYC', 'HomeRC', 'AwayRC']

test_df = pd.DataFrame( columns = colNames)

In [128]:
test_df

,HomeGoals,AwayGoals,HomeXG,AwayXG,HomeShots,AwayShots,HomeCorners,AwayCorners,HomeSubs,AwaySubs,HomeYC,AwayYC,HomeRC,AwayRC


In [ ]:
tempArray = np.zeros((25,14))
#tempArray

In [139]:
tempString = np.full((25, 2), ['                                       ', '                                       '])
#tempString

Ran into a slight problem. Can't append a dataFrame the way I want to, so going to use arrays. But can only store one data type in an array. So going to make a temp array for the name of the teams, and another one for the numeric data.

In [140]:
tempHomeTeam = ' '
tempAwayTeam = ' '
numGames = 0
boolGame = False

for i in range(tempData.shape[0]):
    # Special case for first row
    if i == 0:
        if(tempData['side'][i] == 1):
            tempHomeTeam = tempData['event_team'][i]
            tempAwayTeam = tempData['opponent'][i]
        else:
            tempAwayTeam = tempData['event_team'][i]
            tempHomeTeam = tempData['opponent'][i]
        # Now assign values
        tempString[0][0] = tempHomeTeam
        tempString[0][1] = tempAwayTeam
        tempArray[0][0] = 0
        tempArray[0][1] = tempData['is_goal'][i]
        tempArray[0][2] = 0
        tempArray[0][3] = tempData['xG'][i]
        tempArray[0][4] = 0
        tempArray[0][5] = tempData['is_shot'][i]
        tempArray[0][6] = 0
        tempArray[0][7] = 0
        tempArray[0][8] = 0
        tempArray[0][9] = 0
        tempArray[0][10] = 0
        tempArray[0][11] = 0
        tempArray[0][12] = 0
        tempArray[0][13] = 0
        
        
        
        
    
    else:
        # Figure out if same homeTeam and awayTeam
        boolGame = (((tempData['side'][i] == 1) & ((tempHomeTeam == tempData['event_team'][i])&(tempAwayTeam == tempData['opponent'][i])))|
               ((tempData['side'][i] == 2) & ((tempAwayTeam == tempData['event_team'][i])&(tempHomeTeam == tempData['opponent'][i]))))
        
        # If in same game, Add current row to 
        if(boolGame):
            if(tempData['side'][i] == 1): # Home team event
                tempArray[numGames][0] += tempData['is_goal'][i]
                tempArray[numGames][2] += tempData['xG'][i]
                tempArray[numGames][4] += tempData['is_shot'][i]
                tempArray[numGames][6] += tempData['is_corner'][i]
                
                if(tempData['event_type'][i] == 7):# Event is a sub
                    tempArray[numGames][8] += 1
                    
                if(tempData['event_type'][i] == 4):# Event is a YC
                    tempArray[numGames][10] += 1
                
                if(tempData['event_type'][i] == 5|tempData['event_type'][i] == 6):# Event is a second YC or a RC
                    tempArray[numGames][12] += 1

                    
            else: # Away Team Event
                tempArray[numGames][1] += tempData['is_goal'][i]
                tempArray[numGames][3] += tempData['xG'][i]
                tempArray[numGames][5] += tempData['is_shot'][i]
                tempArray[numGames][7] += tempData['is_corner'][i]
                
                if(tempData['event_type'][i] == 7):# Event is a sub
                    tempArray[numGames][9] += 1
                    
                if(tempData['event_type'][i] == 4):# Event is a YC
                    tempArray[numGames][11] += 1
                
                if(tempData['event_type'][i] == 5|tempData['event_type'][i] == 6):# Event is a second YC or a RC
                    tempArray[numGames][13] += 1
                
            
            
        # If not in same game, Change the home team and away team
        else:
            # Add one to the number of games
            # This will effectively add the next row 
            numGames +=1
            # Get name of new teams and Add them to the array of team Names
            if(tempData['side'][i] == 1):
                tempHomeTeam = tempData['event_team'][i]
                tempAwayTeam = tempData['opponent'][i]
                tempString[numGames][0] = tempHomeTeam
                tempString[numGames][1] = tempAwayTeam
            else:
                tempAwayTeam = tempData['event_team'][i]
                tempHomeTeam = tempData['opponent'][i]
                tempString[numGames][0] = tempHomeTeam
                tempString[numGames][1] = tempAwayTeam

            # Then create new row and start adding onto that 
            if(tempData['side'][i] == 1): # Home team event
                tempArray[numGames][0] += tempData['is_goal'][i]
                tempArray[numGames][2] += tempData['xG'][i]
                tempArray[numGames][4] += tempData['is_shot'][i]
                tempArray[numGames][6] += tempData['is_corner'][i]
                
                if(tempData['event_type'][i] == 7):# Event is a sub
                    tempArray[numGames][8] += 1
                    
                if(tempData['event_type'][i] == 4):# Event is a YC
                    tempArray[numGames][10] += 1
                
                if(tempData['event_type'][i] == 5|tempData['event_type'][i] == 6):# Event is a second YC or a RC
                    tempArray[numGames][12] += 1

                    
            else: # Away Team Event
                tempArray[numGames][1] += tempData['is_goal'][i]
                tempArray[numGames][3] += tempData['xG'][i]
                tempArray[numGames][5] += tempData['is_shot'][i]
                tempArray[numGames][7] += tempData['is_corner'][i]
                
                if(tempData['event_type'][i] == 7):# Event is a sub
                    tempArray[numGames][9] += 1
                    
                if(tempData['event_type'][i] == 4):# Event is a YC
                    tempArray[numGames][11] += 1
                
                if(tempData['event_type'][i] == 5|tempData['event_type'][i] == 6):# Event is a second YC or a RC
                    tempArray[numGames][13] += 1

In [143]:
tempArray

array([[ 3.        ,  1.        ,  1.78865546,  0.44570136, 13.        ,
         4.        ,  7.        ,  7.        ,  3.        ,  3.        ,
         1.        ,  3.        ,  0.        ,  0.        ],
       [ 2.        ,  2.        ,  1.48710407,  0.60925986, 10.        ,
         6.        ,  5.        ,  4.        ,  3.        ,  2.        ,
         2.        ,  4.        ,  0.        ,  0.        ],
       [ 2.        ,  0.        ,  2.08472851,  0.88133484, 10.        ,
         6.        ,  6.        ,  5.        ,  2.        ,  3.        ,
         1.        ,  2.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.63914027,  0.70267722,  9.        ,
         7.        ,  7.        ,  6.        ,  3.        ,  3.        ,
         1.        ,  3.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.77860375,  1.64204374,  9.        ,
         9.        ,  3.        , 11.        ,  2.        ,  2.        ,
         2.        ,  1.  

Okay, Have the general functionality that I want. Now need to do it over the entire original. Hecking sweet my doggie

Going to need to make the two Arrays quite large. Thinking something like 10,000 just to give myself an extra 500 or so to make sure there is enough room in the arrays.

In [4]:
# Re-read in the data
data = pd.read_csv(path)

Drop some columns and rows that are events I don't currently care about. 

Note: Taking out fouls almost halfed the dataset

In [21]:
data = data.drop(['id_odsp','id_event', 'sort_order'], axis=1)

In [5]:
print(data.shape)
# This deletes wherever shot_outcome = 3
tempData = data.loc[data["shot_outcome"] != 3]
data = tempData
# tempData = testData.loc[testData["location"] != 19]
# testData = tempData
tempData = data.loc[data["event_type"] != 0]
data = tempData
tempData = data.loc[data["event_type"] != 3]
data = tempData
tempData = data.loc[data["event_type"] != 8]
data = tempData
tempData = data.loc[data["event_type"] != 9]
data = tempData
tempData = data.loc[data["event_type"] != 10]
data = tempData
tempData = data.loc[data["event_type"] != 11]
data = tempData
print(data.shape)

(941009, 22)
(359158, 22)


Reset the Index for the data after removing the rows

In [6]:
# Reset the index
data = data.reset_index(drop = True)

In [7]:
data.head(3)

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0


Take just the shot data. And then reset the index for the shot dataset

In [8]:
shotData = data.loc[data["event_type"] == 1]
shotData = shotData.reset_index(drop=True)
shotData.head(3)

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit12,12,14,Attempt missed. Shinji Kagawa (Borussia Dortmu...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,13.0,2.0,0,15.0,1.0,1,1.0,0
2,UFot0hit/,UFot0hit14,14,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,4.0,1.0,1,9.0,2.0,1,1.0,0


Now find the xG values for every zone

Note: Zone 1 and 2 don't have any shots in them. They are attacking half and defensive half. So used for broad foul location

In [9]:
numShots = list()
numGoals = list()
xG = list()

In [10]:
countShots = 0
countGoals = 0

for j in range(3,20):
    countShots = 0
    countGoals = 0
    
    for i in range(shotData.shape[0]):
        if (shotData['location'][i] == j) & (shotData['is_goal'][i] == 0):
            countShots += 1
        if (shotData['location'][i] == j) & (shotData['is_goal'][i] == 1):
            countShots += 1
            countGoals += 1
            
    numShots.append(countShots)
    numGoals.append(countGoals)

In [11]:
for i in range(len(numShots)):
    if(numShots[i] != 0):
        xG.append(numGoals[i]/numShots[i])
    else:
        xG.append(0)


In [12]:
countShots = 0
countGoals = 0
for i in range(len(numShots)):
    countShots += numShots[i]
    countGoals += numGoals[i]

print(countShots)
print(countGoals)

175053
24441


Assign the overall xG for all shots to zone 19 because it is unkown. Then round the xG values to 4 decimals

In [13]:
xG[16] = countGoals / countShots
xG_rounded = [round(num, 4) for num in xG]

In [14]:
print(xG_rounded)

[0.1885, 0, 0, 0.0341, 0.0926, 0.0894, 0.0887, 0.23, 0.0936, 0.2369, 0.5363, 0.7662, 0.0391, 0.0189, 0.0136, 0.0163, 0.1396]


In [15]:
data['xG'] = 0#testData['player_in'][0] # nan
data['is_shot'] = 0
data['is_corner'] = 0
data.head(3)

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,xG,is_shot,is_corner
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,2.0,0,9.0,2.0,1,1.0,0,0,0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,0,NaN,NaN,0,NaN,0,0,0,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,0,NaN,NaN,0,NaN,0,0,0,0


Now Assign the xG values, and fill in the boolean values for is_shot and is_goal

Note: This should take up to 50 minutes

In [17]:
for i in range(data.shape[0]):
    if (data['event_type'][i]==1):
        data['is_shot'][i] = 1
        
        for j in range(len(numShots)):
            if(data['event_type'][i]==1) & (data['location'][i] == j+3):
                data['xG'][i] = xG_rounded[j]
            
    if(data['event_type'][i]==2):
        data['is_corner'][i] = 1

C:\Users\Joe Yando\AppData\Local\Temp\ipykernel_5388\4168963375.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_shot'][i] = 1
C:\Users\Joe Yando\AppData\Local\Temp\ipykernel_5388\4168963375.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['xG'][i] = xG_rounded[j]
C:\Users\Joe Yando\AppData\Local\Temp\ipykernel_5388\4168963375.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_corner'][i] = 1


In [22]:
data.head(10)

,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,...,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,xG,is_shot,is_corner
0,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,NaN,...,2.0,0,9.0,2.0,1,1.0,0,0.0887,1,0
1,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.0000,0,1
2,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.0000,0,1
3,10,"Corner, Hamburg. Conceded by Lukasz Piszczek.",2,NaN,2,Hamburg SV,Borussia Dortmund,lukasz piszczek,lukasz piszczek,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.0000,0,1
4,14,Attempt missed. Shinji Kagawa (Borussia Dortmu...,1,12.0,1,Borussia Dortmund,Hamburg SV,shinji kagawa,mario gotze,NaN,...,2.0,0,15.0,1.0,1,1.0,0,0.0391,1,0
5,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,kevin grosskreutz,mario gotze,NaN,...,1.0,1,9.0,2.0,1,1.0,0,0.0887,1,0
6,25,Shinji Kagawa (Borussia Dortmund) hits the rig...,1,12.0,1,Borussia Dortmund,Hamburg SV,shinji kagawa,lukasz piszczek,NaN,...,4.0,0,3.0,1.0,1,1.0,0,0.1885,1,0
7,25,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.0000,0,1
8,25,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,NaN,...,NaN,0,NaN,NaN,0,NaN,0,0.0000,0,1
9,26,Attempt saved. Mats Hummels (Borussia Dortmund...,1,12.0,1,Borussia Dortmund,Hamburg SV,mats hummels,chris lowe,NaN,...,1.0,0,3.0,3.0,2,1.0,0,0.1885,1,0


Now making the arrays and will append to them for each game

In [63]:
tempArray = np.zeros((10000,14))
#tempArray
tempString = np.full((10000, 2), ['                                       ', '                                       '])
#tempString

Now iterate over the whole dang thang and get some sweet results

In [64]:
tempHomeTeam = ' '
tempAwayTeam = ' '
numGames = 0
boolGame = False

for i in range(data.shape[0]):
    # Special case for first row
    if i == 0:
        if(data['side'][i] == 1):
            tempHomeTeam = data['event_team'][i]
            tempAwayTeam = data['opponent'][i]
        else:
            tempAwayTeam = data['event_team'][i]
            tempHomeTeam = data['opponent'][i]
        # Now assign values
        tempString[0][0] = tempHomeTeam
        tempString[0][1] = tempAwayTeam
        tempArray[0][0] = 0
        tempArray[0][1] = data['is_goal'][i]
        tempArray[0][2] = 0
        tempArray[0][3] = data['xG'][i]
        tempArray[0][4] = 0
        tempArray[0][5] = data['is_shot'][i]
        tempArray[0][6] = 0
        tempArray[0][7] = 0
        tempArray[0][8] = 0
        tempArray[0][9] = 0
        tempArray[0][10] = 0
        tempArray[0][11] = 0
        tempArray[0][12] = 0
        tempArray[0][13] = 0
          
    
    else:
        # Figure out if same homeTeam and awayTeam
        boolGame = (((data['side'][i] == 1) & ((tempHomeTeam == data['event_team'][i])&(tempAwayTeam == data['opponent'][i])))|
               ((data['side'][i] == 2) & ((tempAwayTeam == data['event_team'][i])&(tempHomeTeam == data['opponent'][i]))))
        
        # If in same game, Add current row to 
        if(boolGame):
            if(data['side'][i] == 1): # Home team event
                tempArray[numGames][0] += data['is_goal'][i]
                tempArray[numGames][2] += data['xG'][i]
                tempArray[numGames][4] += data['is_shot'][i]
                tempArray[numGames][6] += data['is_corner'][i]
                
                if(data['event_type'][i] == 7):# Event is a sub
                    tempArray[numGames][8] += 1
                    
                if(data['event_type'][i] == 4):# Event is a YC
                    tempArray[numGames][10] += 1
                
                if(data['event_type'][i] == 5):# Event is a second YC or a RC
                    tempArray[numGames][12] += 1
                if(data['event_type'][i] == 6):# Event is a second YC or a RC
                    tempArray[numGames][12] += 1

                    
            else: # Away Team Event
                tempArray[numGames][1] += data['is_goal'][i]
                tempArray[numGames][3] += data['xG'][i]
                tempArray[numGames][5] += data['is_shot'][i]
                tempArray[numGames][7] += data['is_corner'][i]
                
                if(data['event_type'][i] == 7):# Event is a sub
                    tempArray[numGames][9] += 1
                    
                if(data['event_type'][i] == 4):# Event is a YC
                    tempArray[numGames][11] += 1
                
                if(data['event_type'][i] == 5):# Event is a second YC or a RC
                    tempArray[numGames][13] += 1
                if(data['event_type'][i] == 6):
                    tempArray[numGames][13] += 1
                
            
            
        # If not in same game, Change the home team and away team
        else:
            # Add one to the number of games
            # This will effectively add the next row 
            numGames +=1
            # Get name of new teams and Add them to the array of team Names
            if(data['side'][i] == 1):
                tempHomeTeam = data['event_team'][i]
                tempAwayTeam = data['opponent'][i]
                tempString[numGames][0] = tempHomeTeam
                tempString[numGames][1] = tempAwayTeam
            else:
                tempAwayTeam = data['event_team'][i]
                tempHomeTeam = data['opponent'][i]
                tempString[numGames][0] = tempHomeTeam
                tempString[numGames][1] = tempAwayTeam

            # Then create new row and start adding onto that 
            if(data['side'][i] == 1): # Home team event
                tempArray[numGames][0] += data['is_goal'][i]
                tempArray[numGames][2] += data['xG'][i]
                tempArray[numGames][4] += data['is_shot'][i]
                tempArray[numGames][6] += data['is_corner'][i]
                
                if(data['event_type'][i] == 7):# Event is a sub
                    tempArray[numGames][8] += 1
                    
                if(data['event_type'][i] == 4):# Event is a YC
                    tempArray[numGames][10] += 1
                
                if(data['event_type'][i] == 5):# Event is a second YC or a RC
                    tempArray[numGames][12] += 1
                if(data['event_type'][i] == 6):
                    tempArray[numGames][12] += 1

                    
            else: # Away Team Event
                tempArray[numGames][1] += data['is_goal'][i]
                tempArray[numGames][3] += data['xG'][i]
                tempArray[numGames][5] += data['is_shot'][i]
                tempArray[numGames][7] += data['is_corner'][i]
                
                if(data['event_type'][i] == 7):# Event is a sub
                    tempArray[numGames][9] += 1
                    
                if(data['event_type'][i] == 4):# Event is a YC
                    tempArray[numGames][11] += 1
                
                if(data['event_type'][i] == 5):# Event is a second YC or a RC
                    tempArray[numGames][13] += 1
                if(data['event_type'][i] == 6):
                    tempArray[numGames][13] += 1

In [65]:
tempArray[9073]

array([ 1.    ,  0.    ,  2.0668,  0.7003, 10.    ,  9.    ,  5.    ,
        4.    ,  3.    ,  3.    ,  1.    ,  3.    ,  0.    ,  0.    ])

In [66]:
tempString[9074]

array(['                                       ',
       '                                       '], dtype='<U39')

So it appears the last game happens at entry 9073

Now convert the arrays to dataframes, cut out the last 1000ish rows, and merge them

In [89]:
#colNames = ['HomeTeam', 'AwayTeam', 'HomeGoals', 'AwayGoals', 'HomeXG', 'AwayXG', 'HomeShots', 'AwayShots','HomeCorners', 
            #'AwayCorners', 'HomeSubs','AwaySubs', 'HomeYC', 'AwayYC', 'HomeRC', 'AwayRC']

colNamesTeams = ['HomeTeam', 'AwayTeam']
    
colNamesStat = ['HomeGoals', 'AwayGoals', 'HomeXG', 'AwayXG', 'HomeShots', 'AwayShots', 'HomeCorners', 'AwayCorners', 'HomeSubs', 
            'AwaySubs', 'HomeYC', 'AwayYC', 'HomeRC', 'AwayRC']

dfTeams = pd.DataFrame(tempString, columns = colNamesTeams)
dfStats = pd.DataFrame(tempArray, columns = colNamesStat)

In [90]:
dfTeams = dfTeams.iloc[:9074]
dfStats = dfStats.iloc[:9074]

In [91]:
dfTeams

,HomeTeam,AwayTeam
0,Borussia Dortmund,Hamburg SV
1,FC Augsburg,SC Freiburg
2,Werder Bremen,Kaiserslautern
3,Paris Saint-Germain,Lorient
4,Caen,Valenciennes
...,...,...
9069,Genoa,Crotone
9070,Chelsea,Hull
9071,AS Monaco,Lorient
9072,Real Betis,Sporting Gijon


In [92]:
dfFull = pd.concat([dfTeams, dfStats], axis = 1, join = 'inner')
dfFull

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomeXG,AwayXG,HomeShots,AwayShots,HomeCorners,AwayCorners,HomeSubs,AwaySubs,HomeYC,AwayYC,HomeRC,AwayRC
0,Borussia Dortmund,Hamburg SV,3.0,1.0,1.8354,0.4050,13.0,4.0,7.0,7.0,3.0,3.0,1.0,3.0,0.0,0.0
1,FC Augsburg,SC Freiburg,2.0,2.0,1.6848,0.5423,10.0,6.0,5.0,4.0,3.0,2.0,2.0,4.0,0.0,0.0
2,Werder Bremen,Kaiserslautern,2.0,0.0,2.5788,1.0306,10.0,6.0,6.0,5.0,2.0,3.0,1.0,2.0,0.0,0.0
3,Paris Saint-Germain,Lorient,0.0,1.0,0.5509,0.8049,9.0,7.0,7.0,6.0,3.0,3.0,1.0,3.0,0.0,0.0
4,Caen,Valenciennes,1.0,0.0,0.7052,1.5982,9.0,9.0,3.0,11.0,2.0,2.0,2.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9069,Genoa,Crotone,2.0,2.0,1.7186,1.6280,5.0,11.0,7.0,5.0,3.0,1.0,1.0,2.0,0.0,0.0
9070,Chelsea,Hull,2.0,0.0,0.9357,0.5653,6.0,8.0,9.0,7.0,3.0,3.0,1.0,3.0,0.0,0.0
9071,AS Monaco,Lorient,4.0,0.0,2.0934,0.5224,15.0,6.0,3.0,5.0,3.0,3.0,1.0,1.0,0.0,0.0
9072,Real Betis,Sporting Gijon,0.0,0.0,1.3234,0.5439,10.0,5.0,9.0,5.0,3.0,3.0,2.0,2.0,0.0,0.0


Now save the File to a csv File

In [93]:
dfFull.to_csv(r'C:\Users\Joe Yando\Desktop\CS-254\finalProject\fullGames.csv',index=False, header = True)